#Tutorial: Recurrent network model (LSTM)

In this tutorial we train a long-short term memory (LSTM) recurrent model to predict the next letter in a sequence of letters, where the order of letters presented and historical context is necessary to predict the output.  

#Background - Element state and historical context 

In a simple way, LSTM networks have internal contextual state and a collection of 'gates' that regulate how they behave as long-term or short-term memory cells.

The output of the LSTM network is modulated by the state of these gates. This is a very important property, that when we need the prediction of the neural network to depend on the historical context of inputs, rather than only on the very last input.


![LSTM Rollout](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)


As a simple example, consider that we want to predict the next number of the following sequence: 6 -> 7 -> 8 -> ?. We would like to have the next output to be 9 (x+1). However, if we provide this sequence: 2 -> 4 -> 8 -> ?, we would like to get 16 (2x).

Although in both cases, the current last input was number 8, the prediction outcome should be different (when we take into account the contextual information of previous values and not only the last one).

[Content borrowed from:  https://medium.com/datathings/the-magic-of-lstm-neural-networks-6775e8b540cd ]

[Image linked from: http://colah.github.io/posts/2015-08-Understanding-LSTMs ]

#Step 0 - Setup the learning environment
Set the batch size to something reasonably small (<< number of songs) and use a larger number of epochs given that the process only has eight 'song' sequences to look at.  NB: LSTMs are stateful, however the Keras implementation of LSTMs resets the state of the network after each batch.

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers


np.random.seed(2)  # for consistancy

batch_size = 4
epochs = 100

print('Environment is ready!')
print(tf.VERSION)
print(tf.keras.__version__)

#Step 1 - Define and prepare the data
The task is to learn sequences called 'songs' that can be simply thought of as a progression of parts (say chords) in a song, represented by alphabet characeters.  The letters will actually be encoded as integer alphabet indices to make them compatible with the LSTM model.  The final letter in each song is the training label.  

Note the **repeating patterns** and similarities between different 'songs'.  For example, songs with labels M and Q ***only differ in two elements***, which occur at very beginning of their sequences.  N is also very similar to these with only a small temporal offest in the repeating pattern. 

In [0]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
song = [
    ['X', 'X', 'A', 'H', 'F', 'E', 'C', 'E', 'B', 'M'],
    ['Y', 'Y', 'Y', 'A', 'H', 'F', 'E', 'C', 'E', 'N'],
    ['F', 'E', 'F', 'C', 'D', 'E', 'F', 'D', 'B', 'O'],
    ['F', 'F', 'F', 'G', 'H', 'G', 'G', 'F', 'E', 'P'],
    ['C', 'B', 'A', 'H', 'F', 'E', 'C', 'E', 'B', 'Q'],
    ['Z', 'X', 'B', 'A', 'B', 'C', 'D', 'E', 'F', 'R'],
    ['W', 'F', 'G', 'H', 'H', 'H', 'G', 'F', 'E', 'L'],
    ['W', 'X', 'Y', 'Z', 'D', 'E', 'F', 'G', 'H', 'K']]

# Translate songs into alphabet indices, put into 'data'
data = []
for row in song:
    # Get the alphabet index for each element the row 
    data.append([alphabet.find(el) for el in row])
data = np.array(data)

# Assume last column as labels and one-hot encode these
labels = data[:, -1]
labels = tf.keras.utils.to_categorical(labels, 26)

# Trim the labels (final column) from the data 
songs = data[:, :-1]

print(songs)

# Reshape into the form [samples(8), time steps(9), features(1)]
songs = np.reshape(songs, [songs.shape[0], songs.shape[1], 1])
   

#Step 2 - Define model architecture

In [0]:
model = tf.keras.Sequential()
model.add(layers.LSTM(64, return_sequences=True, input_shape=(songs.shape[1], songs.shape[2])))
model.add(layers.LSTM(32))
model.add(layers.Dense(len(alphabet), activation='softmax'))

# Using stochastic gradient decent with two parameters
sgd = tf.keras.optimizers.SGD(lr=0.1, momentum=0.9)

# Print a quick summary of the model to verify our work
model.summary()

# Compile using some 'typical' parameters for loss, optimizer and error metric
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])



#Step 3 - Train the model

In [0]:
model.fit(songs, labels, batch_size, epochs, shuffle=True)
scores = model.evaluate(songs, labels)

print('Accuracy: {0:2.2}'.format(scores[1]))


#Step 4 - Print some sample results

In [0]:
r = 8

for i in range(r):
    # Choose an individual song at random and format it for testing
    #rand_idx = np.random.randint(0, songs.shape[0])
    test_song  = songs[i] 
    model_input = np.reshape(test_song, [1, 9, 1])
    
    # Given the sequence, predict the next letter
    prediction = model.predict(model_input)
    
    # Decode and print the sequence followed by the predicted next letter
    letter_in = []
    for el in test_song:
        letter_in.append(alphabet[el[0]])
    if np.argmax(prediction) == np.argmax(labels[i]):
        msg='Correct!'
    else:
        msg='Incorrect'
    print("{0} -> {1} ... {2}".format(letter_in, alphabet[np.argmax(prediction)], msg))
   

#Step 5 - Testing for robustness / generalization 
Using testsong, with slight changes to the elements of the input sequences, how does the LSTM perform? 

Consider the following changes: 
![testsong_image](https://aida.acadiau.ca/tl_files/sites/aida/test_song.png)

[testsong](https://drive.google.com/open?id=1rGsyA34QAdQYF8a8rHMtjafM6AKxdGrs)



In [0]:
testsong = [
    ['X', 'X', 'A', 'H', 'G', 'E', 'C', 'E', 'B', 'M'],
    ['Y', 'Y', 'Y', 'B', 'H', 'F', 'F', 'C', 'E', 'N'],
    ['F', 'E', 'F', 'C', 'D', 'E', 'F', 'D', 'B', 'O'],
    ['F', 'F', 'F', 'G', 'G', 'G', 'G', 'F', 'O', 'P'],
    ['C', 'B', 'A', 'H', 'F', 'E', 'D', 'E', 'B', 'Q'],
    ['Z', 'X', 'B', 'A', 'B', 'E', 'E', 'E', 'F', 'R'],
    ['W', 'F', 'G', 'H', 'H', 'H', 'H', 'F', 'E', 'L'],
    ['W', 'X', 'Y', 'Z', 'D', 'E', 'F', 'G', 'G', 'K']]

# In practice, we would never rewrite a section of code like this!

testdata = []
for row in testsong:
    # Get the alphabet index for each element the row 
    testdata.append([alphabet.find(el) for el in row])
testdata = np.array(testdata)
testlabels = testdata[:, -1]
testlabels = tf.keras.utils.to_categorical(testlabels, 26)
testsongs = testdata[:, :-1]
testsongs = np.reshape(testsongs, [testsongs.shape[0], testsongs.shape[1], 1])

r = 8
for i in range(r):
    test_song  = testsongs[i] 
    model_input = np.reshape(test_song, [1, 9, 1])
    prediction = model.predict(model_input)
    letter_in = []
    for el in test_song:
        letter_in.append(alphabet[el[0]]) 
    if np.argmax(prediction) == np.argmax(testlabels[i]):
        msg='Correct!'
    else:
        msg='Incorrect'
    print("{0} -> {1} ... {2}".format(letter_in, alphabet[np.argmax(prediction)], msg))
    
    

